In [ ]:
from datasets import load_dataset, load_from_disk

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from minbpe import BasicTokenizer

import torch
from torch import nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import  Dataset, DataLoader

from evaluate import load

## Read Data

In [ ]:
ds = load_dataset("abisee/cnn_dailymail", "3.0.0")

# Save to disk
# ds.save_to_disk("data/cnn_dailymail_dataset")

ds

In [2]:
# If you already save to disk
ds = load_from_disk("data/cnn_dailymail_dataset")
ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [3]:
train_data = ds['train'].select_columns(["article", "highlights"])
val_data = ds['validation'].select_columns(["article", "highlights"])
test_data = ds['test'].select_columns(["article", "highlights"])
train_data, val_data, test_data

(Dataset({
     features: ['article', 'highlights'],
     num_rows: 287113
 }),
 Dataset({
     features: ['article', 'highlights'],
     num_rows: 13368
 }),
 Dataset({
     features: ['article', 'highlights'],
     num_rows: 11490
 }))

## Tokenization

### Tokenizer

In [ ]:
# The data used for training
all_articles_text = " ".join(train_data["article"][:1000])
len(all_articles_text)

3530528

In [ ]:
tokenizer = BasicTokenizer()
tokenizer.train(all_articles_text, vocab_size=1024)

In [ ]:
# See how it encode
encoded_article_0 = tokenizer.encode(train_data['article'][0])
encoded_article_0[:5]

[76, 79, 78, 68, 79]

In [26]:
# See how it decode
decoded_article_0 = tokenizer.decode(encoded_article_0)
decoded_article_0, decoded_article_0 == train_data['article'][0]

('LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details 

In [ ]:
# Add special tokens, we do not need <unk> here because data is in english and fit in ASCII
max_vocab_id = list(tokenizer.vocab.keys())[-1]
tokenizer.special_tokens = {
    "<sos>": max_vocab_id + 1,
    "<eos>": max_vocab_id + 2,
    "<unk>": max_vocab_id + 3,
    "<pad>": max_vocab_id + 4,
}

In [ ]:
# See how it works with special tokens
encoded_special_0 = tokenizer.encode(f"<sos> {train_data['article'][0]} <eos> <pad> <pad>")
decoded_special_0 = tokenizer.decode(encoded_special_0)
decoded_special_0, decoded_special_0 == f"<sos> {train_data['article'][0]} <eos> <pad> <pad>"

('<sos> LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. De

In [30]:
# Save to disk
tokenizer.save("model/model_article_1000")

In [4]:
# Load from disk
tokenizer = BasicTokenizer()
tokenizer.load("model/model_article_1000.model")

### Tokenize Data

In [ ]:
def tokenize_fields(example):
    example["article"] = tokenizer.encode(example["article"])
    example["highlights"] = tokenizer.encode(example["highlights"])
    return example

# Tokenize each split
train_tokenized = train_data.select(range(10_000)).map(tokenize_fields)
val_tokenized = val_data.select(range(2500)).map(tokenize_fields)
test_tokenized = test_data.select(range(2500)).map(tokenize_fields)

In [ ]:
# Save to disk
train_tokenized.save_to_disk("data/cnn_train_tokenized_10k")
val_tokenized.save_to_disk("data/cnn_val_tokenized_2500")
test_tokenized.save_to_disk("data/cnn_test_tokenized_2500")

In [5]:
# If you already save to disk
train_tokenized = load_from_disk("data/cnn_train_tokenized_10k")
val_tokenized = load_from_disk("data/cnn_val_tokenized_2500")
test_tokenized = load_from_disk("data/cnn_test_tokenized_2500")

## Vectorizer

In [6]:
class Vectorizer:
  def __init__(self, tokenizer: BasicTokenizer):
    self.tokenizer = tokenizer
    self.vocab_size = len(tokenizer.vocab)
    self.sos_idx = tokenizer.special_tokens["<sos>"]
    self.eos_idx = tokenizer.special_tokens["<eos>"]
    self.pad_idx = tokenizer.special_tokens["<pad>"]

  # Add <sos> and <eos> at the start and end, pad data if necessasry
  def index_vectorize(self, tokens, max_length=1024):
    indices = tokens[:max_length - 2]
    indices = [self.sos_idx] + indices + [self.eos_idx]
    indices += [self.pad_idx] * (max_length - len(indices))
    return indices

In [7]:
article_vectorizer = Vectorizer(tokenizer)

In [61]:
tokenizer.decode(article_vectorizer.index_vectorize(encoded_article_0))

'<sos>LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Deta

## Dataset

In [8]:
class IndexArticleDataset(Dataset):
  def __init__(self, input, target, vectorizer: Vectorizer, max_input_length=1024, max_target_length=128):
    self.input = input
    self.target = target
    self.vectorizer = vectorizer
    
    self.max_input_length = max_input_length
    self.max_target_length = max_target_length
    
    self.sos_index = vectorizer.sos_idx
    self.eos_index = vectorizer.eos_idx
    self.pad_index = vectorizer.pad_idx

  def __len__(self):
    return len(self.input)

  def __getitem__(self, index):
    return {'x': torch.as_tensor(self.vectorizer.index_vectorize(self.input[index], self.max_input_length)),
            'y': torch.as_tensor(self.vectorizer.index_vectorize(self.target[index], self.max_target_length))}

  def get_vectorizer(self):
    return self.vectorizer 
  
  def get_num_batches(self, batch_size):
    return len(self) // batch_size 

In [9]:
train_dataset = IndexArticleDataset(train_tokenized['article'], 
                                    train_tokenized['highlights'], 
                                    article_vectorizer,
                                    max_input_length=512,
                                    max_target_length=128)

val_dataset = IndexArticleDataset(val_tokenized['article'], 
                                  val_tokenized['highlights'], 
                                  article_vectorizer,
                                  max_input_length=512,
                                  max_target_length=128)

test_dataset = IndexArticleDataset(test_tokenized['article'], 
                                   test_tokenized['highlights'], 
                                   article_vectorizer,
                                   max_input_length=512,
                                   max_target_length=128)

In [85]:
out_dict = train_dataset.__getitem__(0)
len(out_dict['x']), len(out_dict['y'])

(512, 128)

In [87]:
tokenizer.decode(out_dict['x'].numpy()), tokenizer.decode(out_dict['y'].numpy())

('<sos>LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Det

## Dataloader

In [10]:
def generate_batches(dataset, batch_size, 
                     shuffle=True,
                     drop_last=True, 
                     device="cpu"):
  
  dataloader = DataLoader(dataset=dataset, 
                          batch_size=batch_size,
                          shuffle=shuffle, 
                          drop_last=drop_last)

  for data_dict in dataloader:
    out_data_dict = {}
    for name, tensor in data_dict.items():
      out_data_dict[name] = data_dict[name].to(device)
    yield out_data_dict

In [91]:
# Using generate_batches to get 1 sample at a time
for out_dict in generate_batches(dataset=train_dataset, batch_size=32):
  x_data = out_dict['x']
  y_target = out_dict['y']
  print(f"x shape: {x_data.shape}, y shape: {y_target.shape}")
  print(x_data)
  print(y_target)
  break

x shape: torch.Size([32, 512]), y shape: torch.Size([32, 128])
tensor([[1024,   40,   69,  ...,   78,  900, 1025],
        [1024,   40,   84,  ...,  401,  276, 1025],
        [1024,   78,   69,  ..., 1027, 1027, 1027],
        ...,
        [1024,   76,   79,  ...,  735,  373, 1025],
        [1024,  702,   65,  ...,  456,  412, 1025],
        [1024,  651,   32,  ...,  926,  549, 1025]])
tensor([[1024,   69,  115,  ..., 1027, 1027, 1027],
        [1024,   65,   99,  ..., 1027, 1027, 1027],
        [1024,   80,  304,  ...,  331,   67, 1025],
        ...,
        [1024,   82,  265,  ..., 1027, 1027, 1027],
        [1024,   78,   69,  ..., 1027, 1027, 1027],
        [1024,   75,  612,  ...,  334,  473, 1025]])


## Model & Training

In [11]:
SEED = 42

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
class Head(nn.Module):
  def __init__(self, embed_dim, head_dim, max_input_length, dropout=0.5):
    super().__init__()
    self.key = nn.Linear(embed_dim, head_dim, bias=False)
    self.query = nn.Linear(embed_dim, head_dim, bias=False)
    self.value = nn.Linear(embed_dim, head_dim, bias=False)
    self.register_buffer('mask', torch.tril(torch.ones(max_input_length, max_input_length)))
    self.dropout = nn.Dropout(dropout)
  
  def forward(self, x):
    B, T, C = x.shape # (batch_size, time_steps, channels=embed_dim)
    print(x.shape)

    K = self.key(x)
    Q = self.query(x)

    KQ = (Q @ K.transpose(-2, -1)) / (np.sqrt(K.shape[-1]))
    KQ_masked = KQ.masked_fill(self.mask[:T, :T] == 0, float('-inf'))
    KQ_masked_softmaxed = torch.softmax(KQ_masked, dim=-1)
    KQ_masked_softmaxed_dropouted = self.dropout(KQ_masked_softmaxed)

    V = self.value(x)
    out = KQ_masked_softmaxed_dropouted @ V
    return out

In [44]:
max_input_length = 512
max_target_length = 128

embed_dim = 384
vocab_size = len(tokenizer.vocab)

head_dim = 64

dropout = 0.5

model = Head(embed_dim, head_dim, max_input_length, dropout)
embed = nn.Embedding(vocab_size, embed_dim, padding_idx=article_vectorizer.pad_idx)
for out_dict in generate_batches(dataset=train_dataset, batch_size=32):
  output = model(embed(out_dict['x']))
  break
output.shape, output

torch.Size([32, 512, 384])


(torch.Size([32, 512, 64]),
 tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 1.1415e-01, -3.5010e-01, -1.3832e-01,  ..., -1.7253e-01,
           -1.8687e-01, -1.6783e-01],
          ...,
          [-1.2226e-01,  3.0152e-02, -3.4217e-02,  ...,  7.6638e-02,
            4.0369e-02, -1.7236e-02],
          [-7.0580e-02,  6.7779e-02,  1.5401e-02,  ...,  2.9139e-02,
            6.9165e-02, -6.1011e-02],
          [-1.3810e-01,  7.4844e-02,  3.8650e-02,  ...,  7.5975e-02,
           -7.4598e-03, -8.3447e-03]],
 
         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 2.3859e-01, -3.8825e-01,  1.1316e-01,  ..., -1.6055e+00,
           -6.7899e-01, -8.7203e-01],
          [-1.3432e-01, -4.1814e-02, -6.1103e-02,  ..., -6.7272e-01,
           -3.5377e-01, -9.35

In [45]:
out_dict['x'].shape

torch.Size([32, 512])